## Import libraries and dataset

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# get dataset
loans_df =  pd.read_csv('../../data/loans_day3.csv', index_col=0)

In [ ]:
#show first 5 lines
loans_df.head()

Note that this is not the dataset that we had in day 1. We have prepared this dataset a bit so that we can better use it in our ML models. This act of treating the data is called `preprocessing`. It was a bit much to fit in one lecture but don't worry.. we've prepared a homework concerning these topics for you!

In [ ]:
#subsample for live code
loans_df = loans_df.sample(50000, random_state=42)

# Regression tasks

Remember with regression tasks we want to predict a continous variable. In our example we're going to try to predict `loan_amnt` with our other features.

### Basic example

Let's code through a simple example and see how we can infere our intercept and slope.

In [ ]:
#sample 50 observations
loans_df_small = loans_df.sample(50)

#split into X and y
X_small = loans_df_small[['annual_inc']]
y_small = loans_df_small['loan_amnt']

In [ ]:
#let's plot our observations as a scatterplot
sns.scatterplot(X_small['annual_inc'], y_small);

In [ ]:
from sklearn.linear_model import LinearRegression

# Instantiate the model
lin_reg = LinearRegression()

# Train the model on the training data
lin_reg.fit(X_small, y_small)

Now that we have a trained model we can use the `.intercept_` and `.coef_` attributes to get our intercept and slope. Then we can plot our line-of-best-fit on our scatterplot.

In [ ]:
intercept = lin_reg.intercept_
slope = lin_reg.coef_

In [ ]:
sns.scatterplot(X_small['annual_inc'], y_small)
plt.plot(X_small['annual_inc'], intercept+slope*X_small, c='r');

### Full Example

#### Train test split

Let's split our data so we can see how well our model generalises. Remember with `generalize` we mean the ability of a model to handle unseen data.

In [ ]:
#define x & y because we don't wanna scale our target
from sklearn.model_selection import train_test_split

X = loans_df.drop(columns=['loan_amnt'])

y = loans_df.loan_amnt

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Linear regression

Linear regression is one of the most basic models but still one of the go-to models in reality.

In [ ]:
from sklearn.linear_model import LinearRegression

# Instantiate the model
lin_reg = LinearRegression()

# Train the model on the training data
lin_reg.fit(X_train, y_train)

Now that we have a trained model, meaning our model knows the ideal intercept and slopes for all features, we can score it using the metrics we saw during the lecture.

We will first have to use the `.predict()` to get our predictions for `X` using our trained model

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, max_error

#get our predictions
y_pred = lin_reg.predict(X_test)

#score our model
lin_mse = mean_squared_error(y_test, y_pred, squared=False)
lin_mae = mean_absolute_error(y_test, y_pred)
lin_max_er = max_error(y_test, y_pred)

# print(f"The MSE for our model is {lin_mse}")
# print(f"The MAE for our model is {lin_mae}")
# print(f"The Max error for our model is {lin_max_er}")

#### Decision tree regressor

Let's do the same process using a `decision tree regressor`. We can then use the metrics to see which model performs better in which case.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Instantiate the model
tree = DecisionTreeRegressor()

# Train the model on the Training data
tree.fit(X_train, y_train)

In [ ]:
#get our predictions
y_pred = tree.predict(X_test)

#score our model
tree_mse = mean_squared_error(y_test, y_pred, squared=False)
tree_mae = mean_absolute_error(y_test, y_pred)
tree_max_er = max_error(y_test, y_pred)

# print(f"The MSE for our model is {tree_mse}")
# print(f"The MAE for our model is {tree_mae}")
# print(f"The Max error for our model is {tree_max_er}")

Now let's compare the two models:

In [ ]:
print(f"RMSE Linear Regression: {round(lin_mse,2)}, RMSE Decision tree {round(tree_mse,2)}")
print(f"MAE Linear Regression: {round(lin_mae,2)}, MAE Decision tree {round(tree_mae,2)}")
print(f"Max error Linear Regression: {round(lin_max_er,2)}, Max error Decision tree {round(tree_max_er,2)}")

We can see that our decision tree model performs better across the board!


## Classification tasks

With classification we're trying to predict a specific class represented by binary values. Let's use an `SVM` and `Decision Tree Classifier` to try to predict `loan_status`. Afterwards we will use the different metrics we learned and see how our models compare to each other.

#### Train test split

In [ ]:
#define x & y because we don't wanna scale our target
from sklearn.model_selection import train_test_split

X = loans_df.drop(columns=['loan_status'])

y = loans_df.loan_status

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### SVM

As mentioned in the lecture we can use a `SVM` for classification tasks. Let's fit, train and score a model on our dateset.

We'll use an `RBF kernel` in our SVM model. This is quite high level for now so what you should understand here is that it allows our model to use higher dimensions for the seperation of the data. Using a `linear kernel` here would be difficult since our data will not be linearly seperable.

Please take note that it can take some time to fit :)

In [ ]:
from sklearn.svm import SVC

# Instantiate the model => we use an rbf kernel to work in higher dimension as this dataset will not be linearly seperable
svm = SVC(kernel='rbf' , class_weight={0:4,1:1})

# Train the model on the Training data
svm.fit(X_train, y_train)

# Score the model on the Test data
svm.score(X_test,y_test)

Let's take a minute to think about our score here. 

The `.score()` method gives back our `accuracy-score`. So our 62% accuracy is the amount of observations our model was right about. Seems relatively good right?

But let's have a look again at the distribution of our `loan_status` status variable.

In [ ]:
# check value counts
y.value_counts()

In [ ]:
# calculate percentage of most dominant class
y.sum() / y.shape[0]

So by calculating the percentage `fully paid` on the total y column, we can see that this class consists of about 80% of the total amount of observations. 

This means that if we would have just guessed this category everytime we would have been right about 80% of the time, which is better than our model.

Let's look into our other metrics to see how our model performs there.

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# calculate y_pred
y_pred = svm.predict(X_test)

#calculate scores
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(accuracy, recall, precision, f1);

Some increases in performance here but still not great. Let's try another model!

#### Decision tree classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

#instantiate model
tree_reg = DecisionTreeClassifier()

#fit model to our training data
tree_reg.fit(X_train, y_train)

#we can use the built-in method .score() to get our accuracy score
tree_reg.score(X_test, y_test)

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

# calculate y_pred
y_pred = tree_reg.predict(X_test)

#calculate scores
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(accuracy, recall, precision, f1);

Looking into our other metrics we see that the model performs a lot better than the SVM one. Still not the score we are looking for though. Let's see tomorrow if we can improve with DL.